# Расчет метрик юнит-экономики с сегментацией когорт

## Анализ датасета

In [1]:
import pandas as pd

In [83]:
df = pd.read_csv('Case for Homework for students - Data.csv')
df

,data,cohort,uuid,buyer,new buyer,aov,cogs,ltc,month,n_payment
0,5/1/2024,2024-05,be239004-bc93-4732-8d2e-db1c9abc70f9,-,False,0,0.0,0,2024-5,NaN
1,5/1/2024,2024-05,ff709093-05bd-4c07-8d47-20a1e22450df,-,False,0,0.0,8,2024-5,NaN
2,5/1/2024,2024-05,024c15c8-33a9-4f4c-b471-c47a2364dc30,-,False,0,0.0,8,2024-5,NaN
3,5/1/2024,2024-05,3186905a-cf69-40fe-a615-51342cc6158b,-,False,0,0.0,8,2024-5,NaN
4,5/1/2024,2024-05,919338ce-b051-4c7a-aa3c-e7c951644a77,-,False,0,0.0,0,2024-5,NaN
...,...,...,...,...,...,...,...,...,...,...
59313,7/31/2024,2024-07,bff9f722-83d9-4907-ac23-cb76399912c5,-,False,0,0.0,8,2024-7,NaN
59314,7/31/2024,2024-07,c5e8d0e7-2128-4113-a29c-866f2abf1af0,-,False,0,0.0,0,2024-7,NaN
59315,7/31/2024,2024-07,d613f8a5-f0f9-4b7b-bc54-27c62bd03eef,-,False,0,0.0,0,2024-7,NaN
59316,7/31/2024,2024-07,6b1105f3-8759-4475-adc8-07447b6a994f,-,False,0,0.0,0,2024-7,NaN


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59318 entries, 0 to 59317
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   data       59318 non-null  object 
 1   cohort     59318 non-null  object 
 2   uuid       59318 non-null  object 
 3   buyer      59318 non-null  object 
 4   new buyer  59318 non-null  bool   
 5   aov        59318 non-null  int64  
 6   cogs       59318 non-null  float64
 7   ltc        59318 non-null  int64  
 8   month      59318 non-null  object 
 9   n_payment  0 non-null      float64
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 4.1+ MB


In [78]:
df.tail()

,data,cohort,uuid,buyer,new buyer,aov,cogs,ltc,month,n_payment
59313,7/31/2024,2024-07,bff9f722-83d9-4907-ac23-cb76399912c5,-,False,0,0.0,8,2024-7,NaN
59314,7/31/2024,2024-07,c5e8d0e7-2128-4113-a29c-866f2abf1af0,-,False,0,0.0,0,2024-7,NaN
59315,7/31/2024,2024-07,d613f8a5-f0f9-4b7b-bc54-27c62bd03eef,-,False,0,0.0,0,2024-7,NaN
59316,7/31/2024,2024-07,6b1105f3-8759-4475-adc8-07447b6a994f,-,False,0,0.0,0,2024-7,NaN
59317,7/31/2024,2024-07,4324a7b7-2781-4694-92d5-8814d886b8e1,-,False,0,0.0,8,2024-7,NaN


In [85]:
cohorts_df = df.groupby('cohort')['uuid'].nunique()

print(cohorts_df, '\n')

unique_uuids = df['uuid'].nunique()
unique_buyers = df[df['buyer'] != '-']['buyer'].nunique()
count_deals = (df[df['buyer'] != '-'].shape[0])

print("Общее количество уникальных uuid:", unique_uuids, '\n',
      "Общее количество уникальных buyers:", unique_buyers, '\n', 
      "Общее количество покупок:",  count_deals, '\n')

cohort
2024-05     9810
2024-06    14547
2024-07    17094
Name: uuid, dtype: int64 

Общее количество уникальных uuid: 41451 
 Общее количество уникальных buyers: 547 
 Общее количество покупок: 745 



## Расчет общей юнит-экономики

In [86]:
# Маркетинговый бюджет
ac_metric = sum(df['ltc'])

# Количество уникальных покупателей
b_metric = unique_buyers

# Общее количество юнитов
ua_metric = unique_uuids

# Количество сделок
t_metric = count_deals

# Средний чек
aov_metric = sum(df['aov'])/b_metric

# Переменные издержки
cogs_metric = sum(df['cogs'])/t_metric

# Стоимость привлечения юнита масштабирования
ltc_cpa_metric = ac_metric/ua_metric

# Конверсия
c1_metric = (b_metric/ua_metric)

# Среднее число сделок на одного клиента
apc_metric = t_metric/b_metric

# Средняя валовая прибыль на клиента
cltv_metric = (aov_metric - cogs_metric) * apc_metric

# Средняя валовая прибыль на юнит масштабирования
ltv_metric = cltv_metric * c1_metric

# Стоимость привлечения клиента
cac_metric = ac_metric/b_metric

# Маржинальная прибыль в юнит-экономике
cm_metric = ua_metric * (ltv_metric - ltc_cpa_metric)

# Выручка
revenue = b_metric * aov_metric * apc_metric

print(f'''AOV: {aov_metric:.2f},
UA: {ua_metric}
COGS: {cogs_metric}
LTC (CPA): {ltc_cpa_metric:.2f}
AC: {ac_metric}
C1: {c1_metric:.2f}
B: {b_metric}
T: {t_metric}
APC: {apc_metric:.2f}
CLTV: {cltv_metric:.2f}
LTV: {ltv_metric:.2f}
CAC: {cac_metric:.2f}
CM: {cm_metric:.2f}
Revenue: {revenue:.2f}
''')

AOV: 13.62,
UA: 41451
COGS: 1.3
LTC (CPA): 7.75
AC: 321365
C1: 0.01
B: 547
T: 745
APC: 1.36
CLTV: 16.78
LTV: 0.22
CAC: 587.50
CM: -312186.79
Revenue: 10146.71



In [87]:
# df['month'] = pd.to_datetime(df['month'], format='%Y-%m', errors='coerce')

## Метрики юнит-экономики по когортам

In [107]:
cohorts = df.groupby('cohort')

cohort_metrics = pd.DataFrame()

# Общее количество юнитов
cohort_metrics['UA'] = cohorts['uuid'].nunique()

# Количество уникальных покупателей
cohort_metrics['B'] = cohorts['buyer'].apply(lambda x: x[x != '-'].nunique())

# Количество сделок
cohort_metrics['T'] = cohorts['buyer'].apply(lambda x: (x != '-').sum())

# Маркетинговый бюджет
cohort_metrics['AC'] = cohorts['ltc'].sum()

# Средний чек
cohort_metrics['AOV'] = cohorts['aov'].sum() / cohort_metrics['B']

# Переменные издержки
cohort_metrics['COGS'] = cohorts['cogs'].sum() / cohort_metrics['T']

# Стоимость привлечения юнита масштабирования
cohort_metrics['LTC_CPA'] = cohort_metrics['AC'] / cohort_metrics['UA']

# Конверсия
cohort_metrics['C1'] = cohort_metrics['B'] / cohort_metrics['UA']

# Среднее число сделок на одного клиента
cohort_metrics['APC'] = cohort_metrics['T'] / cohort_metrics['B']

# Средняя валовая прибыль на клиента
cohort_metrics['CLTV'] = (cohort_metrics['AOV'] - cohort_metrics['COGS']) * cohort_metrics['APC']

# Средняя валовая прибыль на юнит масштабирования
cohort_metrics['LTV'] = cohort_metrics['CLTV'] * cohort_metrics['C1']

# Стоимость привлечения клиента
cohort_metrics['CAC'] = cohort_metrics['AC'] / cohort_metrics['B']

# Маржинальная прибыль в юнит-экономике
cohort_metrics['CM'] = cohort_metrics['UA'] * (cohort_metrics['LTV'] - cohort_metrics['LTC_CPA'])

# Выручка
cohort_metrics['Revenue'] = cohort_metrics['B'] * cohort_metrics['AOV'] * cohort_metrics['APC']

# totals = cohort_metrics.sum()

# cohort_metrics.loc['Total'] = totals

cohort_metrics.round(2)

,UA,B,T,AC,AOV,COGS,LTC_CPA,C1,APC,CLTV,LTV,CAC,CM,Revenue
cohort,,,,,,,,,,,,,,
2024-05,9810,197,328,113352,16.65,1.3,11.55,0.02,1.66,25.56,0.51,575.39,-108317.28,5461.12
2024-06,14547,201,268,118106,13.33,1.3,8.12,0.01,1.33,16.04,0.22,587.59,-114881.07,3573.33
2024-07,17094,149,149,89907,10.00,1.3,5.26,0.01,1.00,8.70,0.08,603.40,-88610.70,1490.00


In [106]:
# есть ли повторяющиеся uuid в разных когортах
multiple_cohorts = df.groupby('uuid')['cohort'].nunique()
print((multiple_cohorts > 1).sum()) 

0


## Метрики юнит-экономики по сегментам

In [108]:
# сколько раз каждый buyer встречается
buyer_counts = df[df['buyer'] != '-']['buyer'].value_counts()

# отбиарем тех, у кого покупок 2 и более
buyers_two_more = buyer_counts[buyer_counts >= 2].index

# Отфильтровываем датафрейм
df_two_more = df[df['buyer'].isin(buyers_two_more)]

In [111]:
cohorts_two_more = df_two_more.groupby('cohort')

cohort_metrics_two_more = pd.DataFrame()

cohort_metrics_two_more['UA'] = cohorts_two_more['uuid'].nunique()

cohort_metrics_two_more['B'] = cohorts_two_more['buyer'].nunique()

cohort_metrics_two_more['T'] = cohorts_two_more.size()

cohort_metrics_two_more['AC'] = cohorts_two_more['ltc'].sum()

cohort_metrics_two_more['AOV'] = cohorts_two_more['aov'].sum() / cohort_metrics_two_more['B']

cohort_metrics_two_more['COGS'] = cohorts_two_more['cogs'].sum() / cohort_metrics_two_more['T']

cohort_metrics_two_more['LTC_CPA'] = cohort_metrics_two_more['AC'] / cohort_metrics_two_more['UA']

cohort_metrics_two_more['C1'] = cohort_metrics_two_more['B'] / cohort_metrics_two_more['UA']

cohort_metrics_two_more['APC'] = cohort_metrics_two_more['T'] / cohort_metrics_two_more['B']

cohort_metrics_two_more['CLTV'] = (cohort_metrics_two_more['AOV'] - cohort_metrics_two_more['COGS']) * cohort_metrics_two_more['APC']

cohort_metrics_two_more['LTV'] = cohort_metrics_two_more['CLTV'] * cohort_metrics_two_more['C1']

cohort_metrics_two_more['CAC'] = cohort_metrics_two_more['AC'] / cohort_metrics_two_more['B']

cohort_metrics_two_more['CM'] = cohort_metrics_two_more['UA'] * (cohort_metrics_two_more['LTV'] - cohort_metrics_two_more['LTC_CPA'])

cohort_metrics_two_more['Revenue'] = cohort_metrics_two_more['B'] * cohort_metrics_two_more['AOV'] * cohort_metrics_two_more['APC']

#  Total (итоги по всем когортам)
# cohort_metrics_two_more.loc['Total'] = cohort_metrics_two_more.sum()

cohort_metrics_two_more.round(2)

,UA,B,T,AC,AOV,COGS,LTC_CPA,C1,APC,CLTV,LTV,CAC,CM,Revenue
cohort,,,,,,,,,,,,,,
2024-05,101,101,232,1224,22.97,1.3,12.12,1.0,2.3,49.78,49.78,12.12,3803.51,5329.11
2024-06,67,67,134,802,20.00,1.3,11.97,1.0,2.0,37.40,37.40,11.97,1703.80,2680.00
